In [10]:
import pandas as pd
import duckdb
import os
import shutil

In [ ]:
file='013356689bd3389f3f29430412edb6a9_2018'

In [12]:
identifier,year=file.split('_')

In [30]:
%%time

extract=duckdb.sql(f'''
select *,
       'POLYGON ((' || X        || ' ' || Y         || ', ' ||
                      (X + 100) || ' ' || Y         || ', ' ||
                      (X + 100) || ' ' || (Y + 100) || ', ' ||
                       X        || ' ' || (Y + 100) || ', ' ||
                       X        || ' ' || Y         || '))' AS wkt_polygon
from (
      select sentinel.X_rounded as X,
             sentinel.Y_rounded as Y,
             sentinel.*,
             codes.*
      from 'parquet_{year}_quantiles/{file}.parquet' as sentinel,
           'corine/clc_benelux_grid_BL.parquet'      as corine,
           'corine/TD_LAND_COVER.parquet'            as codes
      where corine.X=sentinel.X_rounded
      and   corine.Y=sentinel.Y_rounded
      and   corine.VALUE=codes.CD_COVER          
      )
''').to_df()
extract.to_csv(f'visualisations/{file}_statistics_with_clc.csv',sep='|',index=False)

CPU times: user 1.04 s, sys: 52.2 ms, total: 1.09 s
Wall time: 3.96 s


In [29]:
extract.shape

(10000, 61)

In [21]:
duckdb.sql(f'''
            select '{file}'           as file,
                   min(X_rounded)     as min_x, 
                   min(Y_rounded)     as min_y, 
                   max(X_rounded)+100 as max_x, 
                   max(Y_rounded)+100 as max_y
            from extract
''').to_df()

,file,min_x,min_y,max_x,max_y
0,013356689bd3389f3f29430412edb6a9_2018,4050000.0,3110000.0,4060000.0,3120000.0


In [15]:
shutil.copyfile(f'data_{year}/{identifier}/response.tiff',f'visualisations/{file}.tiff')

'visualisations/013356689bd3389f3f29430412edb6a9_2018.tiff'